In [136]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

import timeit

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random


from dateutil.parser import parse

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

from sklearn import neighbors, datasets, tree, svm, preprocessing, utils
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss, classification_report, confusion_matrix, mean_squared_error, accuracy_score

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [397]:
def motorcycle_post_parser(total_listings):
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts'].find()[0:total_listings]
    
    title = []
    year = []
    price = []
    neighborhood = []
    description = []
    url = []
    #attributes
    model = []
    vin = []
    condition = []
    cryptocurrency = []
    delivery = []
    engine_size = []
    fuel_type = []
    mileage = []
    paint_color = []
    street_legal = []
    title_status = []
    transmission = []
    bike_type = []

    iterations = 0
    print('Starting Parsing')
    print(' ')
    for _ in post_html:         
        
        # attributes key words to parse through attributes
        vin_ = 'VIN:'
        condition_ = 'condition:'
        cryptocurrency_ = 'cryptocurrency'
        delivery_ = 'delivery'
        engine_size_ = 'engine displacement'
        fuel_type_ = 'fuel:'
        mileage_ = 'odometer:'
        paint_color_ = 'paint color:'
        street_legal_ = 'street legal'
        title_status_ = 'title status:'
        transmission_ = 'transmission:'
        bike_type_ = 'type:'  
        
        response = _['html']
        soup = BeautifulSoup(response, 'html.parser')

        try:
            post_title = soup.find('span', id = 'titletextonly')
            title.append(post_title.text)
        except:
            title.append(np.nan)
            
        try:
            post_price = soup.find('span', class_ = 'price').text
            price.append(float(post_price.lstrip('$')))
        except:
            price.append(np.nan)
        
        try:
            post_neighborhood = soup.find('small')
            neighborhood.append(((post_neighborhood.text).replace(' (', '')).replace(')', ''))
        except:
            neighborhood.append(np.nan)
        
        try:
            post_description = soup.find('section', id = 'postingbody')
            description.append((post_description.text).replace('\n\nQR Code Link to This Post\n\n\n', ''))
        except:
            description.append(np.nan)
        
        try:
            year_ = parse(post_title.text, fuzzy=True).year
            if 1920 < year_ < 2022:
                year.append(year_)
            else:
                year.append(np.nan)
        except:
            year.append(np.nan)
        
        url.append(_['_id'])
        
        post_attributes = str(soup.find_all('p', attrs = {'class': 'attrgroup'}))
        post_attributes = post_attributes.replace(post_attributes, str(post_attributes.replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''','').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '').replace('''<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]''', '')))
                    
        bike_attributes = (post_attributes.split(','))

        vin_res = [i for i in bike_attributes if vin_ in i]
        condition_res = [i for i in bike_attributes if condition_ in i]
        cryptocurrency_res = [i for i in bike_attributes if cryptocurrency_ in i]
        delivery_res = [i for i in bike_attributes if delivery_ in i]
        engine_size_res = [i for i in bike_attributes if engine_size_ in i]
        fuel_type_res = [i for i in bike_attributes if fuel_type_ in i]
        mileage_res = [i for i in bike_attributes if mileage_ in i]
        paint_color_res = [i for i in bike_attributes if paint_color_ in i]
        street_legal_res = [i for i in bike_attributes if street_legal_ in i]
        title_status_res = [i for i in bike_attributes if title_status_ in i]
        transmission_res = [i for i in bike_attributes if transmission_ in i]
        bike_type_res = [i for i in bike_attributes if bike_type_ in i]
        
        #     
        
        try:
            model.append(bike_attributes[0])
        except:
            model.append('')
                
        try:
            vin.append((vin_res[0]).replace((vin_res[0], '1')))
        except:
            vin.append(0)
            
        try:
            condition.append(str(condition_res[0]).replace('condition:', ''))
        except:
            condition.append('good')
               
        try:
            cryptocurrency.append((cryptocurrency_res[0]).replace((cryptocurrency_res[0]), '1'))
        except:
            cryptocurrency.append(0)
        
        try:
            delivery.append((delivery_res[0]).replace((delivery_res[0]), '1'))
        except:
            delivery.append(0)
                
        try:
            engine_size.append(float(str(engine_size_res[0]).replace('engine displacement (CC):', '')))
        except:
            engine_size.append(np.nan)
            
        try:
            fuel_type.append(str(fuel_type_res[0]).replace('fuel:', ''))
        except:
            fuel_type.append('gas') #gas is the most common
            
        try:
            mileage.append(float(str(mileage_res[0]).replace('odometer:', '')))
        except:
            mileage.append(np.nan)
                    
        try:
            paint_color.append(str(paint_color_res[0]).replace('paint color:', ''))
        except:
            paint_color.append('black') #black is the most common
                 
        try:
            street_legal.append((street_legal_res[0]).replace(street_legal_res[0], '1'))
        except:
            street_legal.append(0)
    
        try:
            title_status.append(str(title_status_res[0]).replace('title status:', ''))
        except:
            title_status.append('clean') #clean is the most common
                
        try: 
            transmission.append(str(transmission_res[0]).replace('transmission:', '')[0:5]) #we are pulling the first five letters to cut out the html
        except:
            transmission.append('manual') 

        try:
            bike_type.append(str(bike_type_res[0]).replace('type:', '')[0:5]) #we are pulling the first five letters to cut out the html
        except:
            bike_type.append('')
                
        iterations += 1

        if iterations%1000 == 0:
            print(str(round(100*round(iterations/total_listings, 2),2)) + '%' + ' parsed succesfully')
    print(' ')
    print('Parsing Complete, Preparing Dataframe')
    df_dictionary = { 'title': title, 'year': year, 'price': price, 'neighborhood': neighborhood, 'description': description, 'url':url, 'model':model, 'vin': vin, 'condition': condition, 'cryptocurrency': cryptocurrency, 'delivery': delivery, 'engine_size': engine_size, 'fuel_type': fuel_type, 'mileage': mileage, 'color': paint_color, 'street_legal': street_legal, 'title_status': title_status, 'transmission': transmission, 'bike_type': bike_type}
    df = pd.DataFrame(df_dictionary)
    
#   Cleaning DataFrame 
    df['neighborhood'].replace('\n       google map\n        ', '', inplace=True)
    
    print(' ')
    print('Parsing Complete! Enjoy Mammal!')
    return df

In [398]:
moto_df = motorcycle_post_parser(58600)

Starting Parsing
 
2.0% parsed succesfully
3.0% parsed succesfully
5.0% parsed succesfully
7.0% parsed succesfully
9.0% parsed succesfully
10.0% parsed succesfully
12.0% parsed succesfully
14.0% parsed succesfully
15.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname V identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


17.0% parsed succesfully
19.0% parsed succesfully
20.0% parsed succesfully
22.0% parsed succesfully
24.0% parsed succesfully
26.0% parsed succesfully
27.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname CB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


29.0% parsed succesfully


...

48.0% parsed succesfully
49.0% parsed succesfully
51.0% parsed succesfully
53.0% parsed succesfully
55.0% parsed succesfully
56.0% parsed succesfully
58.0% parsed succesfully
60.0% parsed succesfully
61.0% parsed succesfully
63.0% parsed succesfully
65.0% parsed succesfully
67.0% parsed succesfully
68.0% parsed succesfully
70.0% parsed succesfully
72.0% parsed succesfully
73.0% parsed succesfully
75.0% parsed succesfully
77.0% parsed succesfully
78.0% parsed succesfully
80.0% parsed succesfully
82.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname D identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


84.0% parsed succesfully
85.0% parsed succesfully
87.0% parsed succesfully
89.0% parsed succesfully
90.0% parsed succesfully
92.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname K identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


94.0% parsed succesfully
96.0% parsed succesfully
97.0% parsed succesfully
99.0% parsed succesfully
 
Parsing Complete, Preparing Dataframe
 
Parsing Complete! Enjoy Mammal!


In [428]:
df = moto_df.copy()

In [400]:
moto_df.to_csv('craigslist_motorcycle_data_full')

In [401]:
df.head()

,title,year,price,neighborhood,description,url,model,vin,condition,cryptocurrency,delivery,engine_size,fuel_type,mileage,color,street_legal,title_status,transmission,bike_type
0,2016 Honda CBR300r,2016.0,2900.0,Auburn,"2016 Honda CBR300r\n6,186 miles as of listing\...",https://auburn.craigslist.org/mcy/d/auburn-uni...,2016 honda cbr300r,0,excellent,0,0,286.0,gas,NaN,yellow,1,clean,manua,sport
1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",NaN,NaN,FAYETTEVILLE,"WE SHIP NATIONWIDE, FINANCE NATIONWIDE! YOU SE...",https://auburn.craigslist.org/mcd/d/fayettevil...,all bikes,0,excellent,0,0,NaN,gas,NaN,black,0,clean,manua,
2,2017 Yamaha FZ 07 very low miles,2017.0,5000.0,Near Montgomery,"2017 Yamaha FZ-07 with only 1100 miles, bike i...",https://auburn.craigslist.org/mcy/d/montgomery...,2017 Yamaha FZ-07,0,like new,0,0,700.0,gas,1100.0,silver,0,clean,manua,stree
3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,NaN,NaN,,2010 Yellow GL1800 CSC Viper\n\nLOOK!!! only 8...,https://auburn.craigslist.org/mcd/d/haleyville...,2010 Honda GL1800 CSC,0,like new,0,0,1832.0,gas,8209.0,yellow,0,clean,manua,trike
4,2006 Honda XR650L *Price Drop*,2006.0,3800.0,Auburn,"2006 Honda XR650L \n\n8,000 miles. This number...",https://auburn.craigslist.org/mcy/d/auburn-200...,2006 honda xr650l,0,excellent,0,0,650.0,gas,8100.0,red,1,clean,manua,dual-


In [402]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58600 entries, 0 to 58599
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           58595 non-null  object 
 1   year            28598 non-null  float64
 2   price           56652 non-null  float64
 3   neighborhood    58483 non-null  object 
 4   description     58595 non-null  object 
 5   url             58600 non-null  object 
 6   model           58600 non-null  object 
 7   vin             58600 non-null  int64  
 8   condition       58600 non-null  object 
 9   cryptocurrency  58600 non-null  object 
 10  delivery        58600 non-null  object 
 11  engine_size     23605 non-null  float64
 12  fuel_type       58600 non-null  object 
 13  mileage         43120 non-null  float64
 14  color           58600 non-null  object 
 15  street_legal    58600 non-null  object 
 16  title_status    58600 non-null  object 
 17  transmission    58600 non-null 

In [429]:
for idx, i in enumerate(df['price']):
    if i > 300000 or i < 100:
        df.drop(idx, axis=0, inplace = True)

In [430]:
df = df.reset_index(drop=True)

In [431]:
for idx, i in enumerate(df['engine_size']):
    if i > 2500:
        df.drop(idx, axis=0, inplace = True)

In [432]:
df = df.reset_index(drop=True)

In [433]:
df['engine_size'].replace(np.nan, df['engine_size'].mean(), inplace=True)

In [434]:
for idx, i in enumerate(df['mileage']):
    if i > 500000:
        df.drop(idx, axis=0, inplace = True)

In [410]:
df = df.reset_index(drop=True)

In [415]:
for idx, i in enumerate(df['condition'][:100]):
    if i == 'new':
        print(idx ,i)
        df['condition'][idx].replace(i, '6')
        print(idx ,i)
    elif i == 'like new':
        print(idx ,i)
        df['condition'][idx].replace(i, '5')
        print(idx ,i)
    elif i == 'excellent':
        print(idx ,i)
        df['condition'][idx].replace(i, '4')
        print(idx ,i)
    elif i == 'good':
        print(idx ,i)
        df['condition'][idx].replace(i, '3')
        print(idx ,i)
    elif i == 'fair':
        print(idx ,i)
        df['condition'][idx].replace(i, '2')
        print(idx ,i)
    elif i == 'salvage':
        print(idx ,i)
        df['condition'][idx].replace(i, '1')
        print(idx ,i)

0 excellent
0 excellent
1 excellent
1 excellent
2 like new
2 like new
3 like new
3 like new
4 excellent
4 excellent
5 excellent
5 excellent
6 like new
6 like new
7 good
7 good
8 like new
8 like new
9 good
9 good
10 excellent
10 excellent
11 like new
11 like new
12 excellent
12 excellent
13 excellent
13 excellent
14 excellent
14 excellent
15 like new
15 like new
16 excellent
16 excellent
17 like new
17 like new
18 like new
18 like new
19 good
19 good
20 like new
20 like new
21 good
21 good
22 good
22 good
23 like new
23 like new
24 good
24 good
25 excellent
25 excellent
26 excellent
26 excellent
27 good
27 good
28 like new
28 like new
29 like new
29 like new
30 like new
30 like new
31 good
31 good
32 good
32 good
33 good
33 good
34 good
34 good
35 good
35 good
36 excellent
36 excellent
37 like new
37 like new
38 excellent
38 excellent
39 good
39 good
40 like new
40 like new
41 like new
41 like new
42 excellent
42 excellent
43 excellent
43 excellent
44 excellent
44 excellent
45 excellent

In [436]:
df['condition'].where(df['condition'] == 'excellent').replace('excellent', '4', inplace=True)

In [437]:
df['condition'][:100]

0     excellent
1     excellent
2      like new
3      like new
4     excellent
        ...    
95     like new
96     like new
97         good
98         good
99         good
Name: condition, Length: 100, dtype: object

In [412]:
df['condition'] = pd.to_numeric(df['condition'])

ValueError: Unable to parse string "excellent" at position 0

In [376]:
df = df.reset_index(drop=True)

In [377]:
df['mileage'].replace(np.nan, df['mileage'].mean(), inplace=True)

In [378]:
df['delivery'] = pd.to_numeric(df['delivery'])

In [379]:
df['street_legal'] = pd.to_numeric(df['street_legal'])

In [381]:
df.dropna(inplace=True)

In [382]:
df = df.reset_index(drop=True)

In [383]:
# condition_dummies = pd.get_dummies(df['condition'])
fuel_dummies = pd.get_dummies(df['fuel_type'])
color_dummies = pd.get_dummies(df['color'])
# title_dummies = pd.get_dummies(df['title_status'])
# transmission_dummies = pd.get_dummies(df['transmission'])
bike_type_dummies = pd.get_dummies(df['bike_type'])
df = pd.concat([df, bike_type_dummies], axis = 1)
# df = pd.concat([df, transmission_dummies], axis = 1)
# df = pd.concat([df, title_dummies], axis = 1)
df = pd.concat([df, color_dummies], axis = 1)
df = pd.concat([df, fuel_dummies], axis = 1)

In [392]:
df.condition.unique()

array(['excellent', 'like new', 'good', 'new', 'fair', 'salvage'],
      dtype=object)

In [391]:
df.title_status.unique()

array(['clean', 'rebuilt', 'lien', 'salvage', 'missing', 'parts only', '',
       'clean<p class="attrgroup">\n<span class="otherpostings">\n<a href="//oklahomacity.craigslist.org/search/sss?userid=336173385">\nmore ads  by this user        </a>\n</span>\n</p>]'],
      dtype=object)

In [388]:
df.to_csv('clean_toy_data')

In [389]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22030 entries, 0 to 22029
Data columns (total 71 columns):
 #   Column                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                         --------------  -----  
 0   title                                                                                                                                                                          22030 non-null  object 
 1   year                                                                                                                                                                           22030 non-null  float64
 2   price                                                                 

In [93]:
df.describe()

,year,price,vin,cryptocurrency,delivery,engine_size,mileage,street legal
count,19945.000000,19945.000000,19945.0,19945.0,19945.0,19945.000000,19945.000000,19945.0
mean,2010.043470,9848.850665,0.0,0.0,0.0,902.339042,15806.002904,0.0
std,9.379696,6287.337136,0.0,0.0,0.0,376.515551,15920.749104,0.0
min,1922.000000,200.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
25%,2006.000000,5499.000000,0.0,0.0,0.0,876.864215,5257.000000,0.0
50%,2013.000000,8500.000000,0.0,0.0,0.0,876.864215,14066.013152,0.0
75%,2016.000000,13000.000000,0.0,0.0,0.0,876.864215,19000.000000,0.0
max,2019.000000,205000.000000,0.0,0.0,0.0,2400.000000,300000.000000,0.0


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           19945 non-null  object 
 1   year            19945 non-null  float64
 2   price           19945 non-null  float64
 3   neighborhood    19945 non-null  object 
 4   description     19945 non-null  object 
 5   url             19945 non-null  object 
 6   model           19945 non-null  object 
 7   vin             19945 non-null  int64  
 8   condition       19945 non-null  object 
 9   cryptocurrency  19945 non-null  int64  
 10  delivery        19945 non-null  int64  
 11  engine_size     19945 non-null  float64
 12  fuel_type       19945 non-null  object 
 13  mileage         19945 non-null  float64
 14  color           19945 non-null  object 
 15  street legal    19945 non-null  int64  
 16  title status    19945 non-null  object 
 17  transmission    19945 non-null 